In [1]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install gymnasium
!pip install gymnasium[classic-control]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import gymnasium as gym
from gym import wrappers
import numpy as np 
import matplotlib.pyplot as plt
import time
import random
from collections import deque

import keras
from keras.activations import relu, linear
from keras.layers import Dense
from keras import Sequential 
from keras.optimizers import Adam

from random import shuffle
import math

In [4]:
ACTION_DIM = 2
STATE_DIM = 4
MAX_ITERATIONS = 200
LEARNING_RATE = 0.01
NUM_EPOCHS = 50

GAMMA = 0.99
REPLAY_MEMORY_SIZE = 10000
NUM_EPISODES = 200
MINIBATCH_SIZE = 32

RANDOM_ACTION_DECAY = 0.99
INITIAL_RANDOM_ACTION = 1
Samples = []
Mean = []

In [5]:
class replay_buffer():
    def __init__(s, max_size):
        s.max_size= max_size
        s.arr = deque() # maxsize = max_size
    
    def add(s, val):
        if len(s.arr) > s.max_size:
            if (np.random.random < 0.5):
                shuffle(s.arr)
            s.arr.popleft()
        s.arr.append(val)
        
    def sample(s, sample_size):
        return random.sample(s.arr, sample_size)

    def __len__(s):
        return len(s.arr)
    
                             
        

In [6]:
def get_q(model, state):
    np_state = np.reshape(state, [-1, STATE_DIM])
    return model.predict(np_state, verbose = 0)
    
def predict(model, state):
    np_state = np.reshape(state, [-1, STATE_DIM]) 
    return model.predict(np_state, verbose = 0)

def train(model, state, targets):
    np_state = np.reshape(state, [-1, STATE_DIM])
    np_targets = np.reshape(targets, [-1, ACTION_DIM])
    model.fit(np_state, np_targets, epochs = 1, verbose = 0)

def get_model():
    model = Sequential()

    model.add(Dense(24, input_shape = (STATE_DIM, ), activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(2, activation = 'linear'))
    
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='mse',
        metrics=[],
    )
    return model

def update_action(action_model, target_model, samples):
    random.shuffle(samples)
    batch_states = []
    batch_targets = []

    for sample in samples:
        old_state, reward, new_state, action = sample
        targets = np.reshape(get_q(action_model, old_state), ACTION_DIM)
        targets[action] = reward
        if new_state is not None:
            predictions = predict(target_model, new_state)
            new_action = np.argmax(predictions)

            targets[action] += GAMMA * predictions[0, new_action]

        batch_states.append(old_state)
        batch_targets.append(targets)

    train(action_model, batch_states, batch_targets)
    


In [7]:
Temp = []

iteration = 0

rand_action_prob = INITIAL_RANDOM_ACTION

replay = replay_buffer(REPLAY_MEMORY_SIZE)

action_model = get_model()

env = gym.make('CartPole-v0')

for episode in range(NUM_EPISODES):
    if np.mean(Temp) > 195 and cnt > 195:
        print('PASSED!')
        break

    rand_action_prob *= RANDOM_ACTION_DECAY
    new_state, info = env.reset(seed = 42)

    for iteration in range(MAX_ITERATIONS):
        rand_action_prob = max(rand_action_prob, 0.1)
        old_state = new_state
        if np.random.random() < rand_action_prob:
            action = env.action_space.sample()
        else:
            q_values = get_q(action_model, old_state)
            sction = q_values.argmax()

        new_state, reward, terminated, done, info = env.step(action)

        if done or terminated:
            Samples.append(iteration+1)
            if (len(Samples) > 50):
                Temp = Samples[-50:] # select most recent 50
                Mean.append(np.mean(Samples[-20:]))
            
            print('episode: ', episode, ' iteration: ', iteration, ' rand_action_prob: ', rand_action_prob)

            if iteration != 199:
                reward = -5
            if iteration == 199:
                reward = 5
            replay.add((old_state, reward, None, action))
            break # and start game again

        replay.add((old_state, reward, new_state, action))

        # update model
        if len(replay) >= MINIBATCH_SIZE and np.random.random() < 0.25 and Samples[-1] < 195:
            minibatch = replay.sample(MINIBATCH_SIZE)
            update_action(action_model, action_model, minibatch)
                
        


env.close()

    

C:\Users\basil\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\basil\AppData\Roaming\Python\Python39\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


episode:  0  iteration:  24  rand_action_prob:  0.99
episode:  1  iteration:  34  rand_action_prob:  0.9801
episode:  2  iteration:  38  rand_action_prob:  0.9702989999999999
episode:  3  iteration:  34  rand_action_prob:  0.96059601
episode:  4  iteration:  15  rand_action_prob:  0.9509900498999999
episode:  5  iteration:  13  rand_action_prob:  0.9414801494009999
episode:  6  iteration:  31  rand_action_prob:  0.9320653479069899
episode:  7  iteration:  19  rand_action_prob:  0.92274469442792
episode:  8  iteration:  10  rand_action_prob:  0.9135172474836407
episode:  9  iteration:  13  rand_action_prob:  0.9043820750088043
episode:  10  iteration:  9  rand_action_prob:  0.8953382542587163
episode:  11  iteration:  16  rand_action_prob:  0.8863848717161291
episode:  12  iteration:  16  rand_action_prob:  0.8775210229989678
episode:  13  iteration:  34  rand_action_prob:  0.8687458127689781
episode:  14  iteration:  36  rand_action_prob:  0.8600583546412883
episode:  15  iteration:  1

KeyboardInterrupt: 

In [ ]:
env = gym.make('CartPole-v1', render_mode = 'human')
Tests = []
for i in range(10):
    prev_state, info = env.reset()
    terminated = False
    done = False
    cnt = 0
    while not terminated and not done:
        q_valus = get_q(action_model, prev_state)
        action = q_values.argmax()
        state, reward, terminated, done, info = env.step(action)
        if done:
            Tests.append(cnt)
            break
        cnt+=1
        prev_state = state
        
env.close()

plt.plot(Tests)
